*The code snippet assumes Anaconda 5.2.0 version of Python virtual environment*

<div class="alert alert-info">
    <h4>Acknowledgement</h4>
    <p>The materials on this post are based the on three NLP papers, <a href="https://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf">Distributed Representations of Words and Phrases and their Compositionality</a> (Mikolov et al., 2013), <a href="https://arxiv.org/pdf/1411.2738.pdf">word2vec Parameter Learning Explained</a> (Rong, 2014) and <a href="https://arxiv.org/pdf/1402.3722.pdf">word2vec Explained: Deriving Mikolov et al.’s
Negative-Sampling Word-Embedding Method</a> (Goldberg and Levy, 2014).</p>
</div>

## Review on Word2Vec Skip-Gram

In my <a href="https://aegis4048.github.io/demystifying_neural_network_in_skip_gram_language_modeling#eq-11">previous post</a>, I illustrated the neural network structure of Skip-Gram Word2Vec model that represents words in a vector space.

<div class="row" id="fig1">
    <div class="col"><img src="jupyter_images/word2vec_skip-gram.png" style="margin: 0;"></div>
    <div class="col-12"><p class="image-description">Figure 1: Skip-Gram model structure</p></div>
</div>

I also derived the cost function of Skip-Gram in <a href="https://aegis4048.github.io/demystifying_neural_network_in_skip_gram_language_modeling#Derivation-of-Cost-Function">Derivation of Cost Function</a>:

<div id="eq-1" style="font-size: 1rem;">
$$J(\theta) = -
\frac{1}{T}
\sum_{t=1}^{T}
\sum_{c=1}^{C}
log 
\frac{exp(W_{output_{(c)}} \cdot h)}{\sum^V_{i=1}exp(W_{output_{(i)}} \cdot h)} \tag{1}$$
</div>

where $T$ is the size of training samples, $C$ is the <a href="https://aegis4048.github.io/demystifying_neural_network_in_skip_gram_language_modeling#Window-Size-of-Skip-Gram">window size</a>, $V$ is the size of unique vocab in the corpus, and $W_{input}$, $W_{output}$ and $h$ are illustrated in <a href="#fig1">figure 1</a>. I also noted that <a href="https://aegis4048.github.io/demystifying_neural_network_in_skip_gram_language_modeling#stochastic">stochastic gradient descent</a> (SGD) is used to mitigate computational burden — the size of $T$ in $\frac{1}{T} \sum^T_{t=1}$ can be billions or more in NLP applications. The new cost function using SGD is:

<div id="eq-2" style="font-size: 1rem;">
$$J(\theta; w^{(t)}) = -
\sum_{c=1}^{C}
log 
\frac{exp(W_{output_{(c)}} \cdot h)}{\sum^V_{i=1}exp(W_{output_{(i)}} \cdot h)} \tag{2}$$
</div>

### Review on Softmax

Softmax is a multinomial regression classifier. It means that it classifies multiple labels, such as predicting if an hand-written digit is $0,\,1,\,2,\,...\,8\,$ or $9$. In case of binary classification (True or False), such as classifying fraud or not-fraud in bank transactions, binomial regression classifier called Sigmoid function is used.

In <a href="#eq-2">eq (2)</a>, the fraction inside the summation of log yields the probability distribution of all $V$-vocabs in the corpus, given the input word. In statistics, the conditional probability of $A$ given $B$ is denoted as $p(A|B)$. In Skip-Gram, we use the notation, $p(w_{context}| w_{center})$, to denote the conditional probability of observing a context word given a center word. It is obtained by using the softmax function:

<div id="eq-3" style="font-size: 1rem;">$$ p(w_{context}|w_{center}) = \frac{exp(W_{output_{(context)}} \cdot h)}{\sum^V_{i=1}exp(W_{output_{(i)}} \cdot h)} \in \mathbb{R}^{1} \tag{3} $$</div>

Exponentiation ensures that the transformed values are positive, and the normalization factor in the denominator ensures that the values have a range of $[0, 1)$ and their sum equals $1$. 

<div class="row give-margin full_screen_margin">
    <div class="col"><img src="jupyter_images/softmax_function.png" style="height: 300px;"></div>
    <div class="col-12"><p class="image-description">Figure 2: softmax function transformation</p></div>
</div>

The probability is computed $V$ times using <a href="#eq-3">eq (3)</a> to obtain a conditional probability distribution of observing all $V$-unique vocabs in the corpus, given a center word ($w^{(t)}$). 

<div id="eq-4" style="font-size: 1rem;">$$ \left[ \begin{array}{c} p(w_{1}|w^{(t)}) \\ p(w_{2}|w^{(t)}) \\ p(w_{3}|w^{(t)}) \\ \vdots \\ p(w_{V}|w^{(t)}) \end{array} \right] = \frac{exp(W_{output} \cdot h)}{\sum^V_{i=1}exp(W_{output_{(i)}} \cdot h)} \in \mathbb{R}^{V}\tag{4} $$</div>

### Softmax is computationally very expensive

There is an issue with the vanilla Skip-Gram — softmax is computationally very expensive, as it requires scanning through the entire output embedding matrix ($W_{output}$) to compute the probability distribution of all $V$ words, where $V$ can be millions or more.

<div class="row give-margin full_screen_margin">
    <div class="col"><img src="jupyter_images/vanilla-skip-gram-complexity.png"></div>
    <div class="col-12"><p class="image-description">Figure 3: Algorithm complexity of vanilla Skip-Gram</p></div>
</div>

Furtheremore, the normalization factor in the denominator also requires $V$ iterations. In mathematical context, the normalization factor needs to be computed for each probability ($w_{context}| w_{center}$), making the alogrithm complexity = $O(V \times V)$. However, when implemented on code, the normalization factor is computed only once and cached as a Python variable, making the alogrithm complexity = $O(V + V) \approx O(V)$. This is possible because normalization factor is the same for all words.

Due to this computational inefficiency, **softmax is not used in most implementaions of Skip-Gram**. Instead we use an alternative called *negative sampling* with <a href="">sigmoid function</a>, which rephrases the problem into a set of independent binary classification task of algorithm complexity = $O(K \, + \, 1)$, where $K$ typically has a range of $[5, 20]$.

## Skip-Gram Negative Sampling

In Skip-Gram, assuming <a href="https://aegis4048.github.io/demystifying_neural_network_in_skip_gram_language_modeling#stochastic">stochastic gradient descent</a>, weight marices in the neural network are updated for each training sample to correctly predict output. Let's assume that the training corpus has 10,000 unique vocabs ($V$ = 10000) and the hidden layer is 300-dimensional ($N$ = 300). This means that there are 3,000,000 neurons in the output weight matrix ($W_{output}$) that need to be updated for each training sample (Notes: for the input weight matrix ($W_{input}$), only 300 neurons are updated for each training sample. This is illustrated in figure 18 of my <a href="https://aegis4048.github.io/demystifying_neural_network_in_skip_gram_language_modeling#weight_update">previous post</a>.) Since the size of the training corpus ($T$) is very large, updating 3M neurons for each training sample is unrealistic in terms of computational efficiency. Negative sampling addresses this issue by updating only a small fraction of the output weight neurons for each training sample. 

In negative sampling, $K$ is the number of <a href="">negative samples</a> randomly drawn from a <a href="">noise distribution</a>. $K$ is a hyper-parameter that can be empirically tuned, with a typical range of $[5,\, 20]$. For each training sample, you randomly draw $K$ number of negative samples from a noise distribution, $P_n(w)$, and the model will update $(k+1) \times N$ neurons in the output weight matrix ($W_{output}$). $N$ is the dimension of the hidden layer ($h$), or the size of a word vector. $+1$ accounts for a <a href="">positive sample</a>. 

With the above assumption, if you set <code>K=9</code>, the model will update (9 + 1) * 300 = 3000 neurons, which is only 0.1% of the 3M neurons in $W_{output}$. This is computationally much faster than the original Skip-Gram, and yet maintains a good quality of word vectors.

The below figure has 3-dimensional hidden layer ($N=3$), 11 vocabs ($V=11$), and 3 negative samples ($K=3$). 

<div class="row" id="fig4">
    <div class="col"><img src="jupyter_images/neg_vs_skip.png" style="margin: 0;"></div>
    <div class="col-12"><p class="image-description">Figure 4: Skip-Gram model structure</p></div>
</div>

<div id="negsample" class="alert alert-info">
    <h4>Notes: Choice of $K$</h4>
    <p>The <a href="https://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf">paper</a> (Mikolov et al., 2013) says that K=2 ~ 5 works for large data sets, and K=5 ~ 20 for small data sets.
    </p>
</div>

### How does negative sampling work?

 With negative sampling, word vectors are no longer learned by predicting context words of a center word. Instead of using softmax to compute the $V$-dimensional probability distribution of observing an output word given an input word, $p(w_O|w_I)$, the model uses sigmoid function to learn to differentiate the actual context words (*positive*) from randomly drawn words (*negative*) from the noise distribution, $P_n(w)$. The idea is that if the model can distinguish between the correct pairs (center, context) vs incorrect pairs (center, randomly drawn negative words), good word vectors will be learned. 

Negative sampling converts multi-classification task into binary-classification task. The new objective is to predict, for any given word-context pair ($w,\,w_c$), whether the word ($w$) is in the context window of the the center word ($w_{center}$) or not. Since the goal is to identify a given word as True (*positive*, $D=1$) or False (*negative*, $D=0$), we use sigmoid function instead of softmax function. The probability can be defined as the following:

<div id="eq-5" style="font-size: 1rem;">
$$
p(D=1|w,w_c;\theta)=\frac{1}{1+exp(-w_{output_{(j)}} \cdot h)}
\in \mathbb{R}^{1}
\tag{5} 
$$</div>

where $w$ is the target word you want to know whether it came from the context window or the noise distribution. $w_c$ is the word vector of the true context word, $h$ is the hidden layer, and $\theta$ is the weight matrix passed into the model. $w_{output_{(j)}}$ is the word vector from the output weight matrix ($W_{output}$). 

<a href="#eq-5">Eq (5)</a> computes the probability that the given word ($w$) is a positive word ($D=1$). It only needs to be applied $K$ + 1 times instead of every word in the vocabulary, because $w_j$ comes from the concatenation of true context word ($w_c$) and $K$-negative samples ($W_{neg} = \{ w_j|j=1,\cdots,K \}$):

<div id="eq-6" style="font-size: 1rem;">
$$
w_{output{(j)}} \in \{w_c\} \cup W_{neg}
\tag{6} 
$$
</div>

This probability is computed $K + 1$ times to obtain a probability distribution of true context word and $K$ negative samples:

<div id="eq-7" style="font-size: 1rem;">
$$
\left[ \begin{array}{c} p(D=1|w_c,w_c) \\ p(D=1|w_{neg, 1},w_c) \\ p(D=1|w_{neg, 2},w_c) \\ p(D=1|w_{neg, 3},w_c) \\ \vdots \\ p(D=1|w_{neg, K},w_c) \end{array} \right] 
= 
\frac{1}{1+exp(-(\{w_c\} \cup W_{neg}) \cdot h)}
\in \mathbb{R}^{k+1}\tag{7} 
$$
</div>

Compare this equation with <a href="#eq-4">eq (4)</a> — you will notice that <a href="#eq-7">eq (7)</a> is computationally much cheaper because $K$ is between 5 ~ 20, whereas $V$ can be millions. Moreover, no extra iterations are necessary to compute the normalization factor in the denominator of <a href="#eq-4">eq (4)</a>, because sigmoid function is a binary regression classifier, and does not need a normalization factor. The computation for probability distribution of vanilla Skip-Gram is $O(V)$, whereas negative sampling's is $O(k+1)$. This shows why negative sampling saves a significant amount of computational cost per iteration.

### Examples

Instead of defining the complete probability distribution over words, the model learns to differentiate between the correct training pairs retrieved from the corpus and the incorrect, randomly generated pairs. For each correct pair the model draws m negative ones. What SGNS does is that it converts this multi-classification problem into binary classification. The new objective of the model is to maximise the probability of the correct samples coming from the corpus and minimise the corpus probability for the negative samples. With NCE, word vectors are no longer learned by attempting to predict the context words from the target word. Instead we learn word vectors by learning how to distinguish true pairs of (target, context) words from corrupted (target, random word from vocabulary) pairs. The idea is that if a model can distinguish between actual pairs of target and context words from random noise, then good word vectors will be learned.

Wneg = {wk~Pw|k = 1, …, K} is the set of “negative words” that are sampled from a frequency distribution Pw over the word vocabulary W, where K is the number of randomly generated words for each context word.


Instead of defining the complete probability distribution over words, the model learns to differentiate between the correct training pairs retrieved from the corpus and the incorrect, randomly generated pairs. For each correct pair the model draws m negative ones. The new objective of the model is to maximise the probability of the correct samples coming from the corpus and minimise the corpus probability for the negative samples

Wneg = {wk~Pw|k = 1, …, K} is the set of “negative words” that are sampled from a frequency distribution Pw over the word vocabulary W, where K is the number of randomly generated words for each context word.

With NCE, word vectors are no longer learned by attempting to predict the context words from the target word. Instead we learn word vectors by learning how to distinguish true pairs of (target, context) words from corrupted (target, random word from vocabulary) pairs. The idea is that if a model can distinguish between actual pairs of target and context words from random noise, then good word vectors will be learned.

What SGNS does is that it converts this multi-classification problem into binary classification. The new objective is to predict, for any given word-context pair (w,c), whether the pair is in the window or not. For this, we try to increase the probability of a “positive” pair (w,c), while at the same time reducing the probability of k randomly chosen “negative samples” (w,s) where s is a word not found in w’s context.



### Choice of negative samples from the noise distribution ($Q$)

#### What is a positive sample?

#### What is a negative sample?

False, or fake sample coming from the noise distribution. Randomly drawn

#### What is noise distribution?

<div><hr></div>
<div class="row give-margin-inline-big-plot">
    <div class="col"><img src="jupyter_images/negative_sample_text.png"></div>
    <div class="col-12"><p class="image-description">Figure 2: softmax function transformation</p></div>
</div>
<div><hr></div>

https://cambridgespark.com/4046-2/


because of this inefficiency most implementations use an alternative, negative-sampling objective, which rephrases the problem as a set of independent binary classification tasks.

Instead of defining the complete probability distribution over words, the model learns to differentiate between the correct training pairs retrieved from the corpus and the incorrect, randomly generated pairs. For each correct pair the model draws m negative ones — with m being a hyperparameter. All negative samples have the same Vt as the original training pair, but their Vc is drawn from an arbitrary noise distribution. Building on the previous example, for the training pair (prickles, nose) the incorrect ones could be (prickles, worked) or (prickles, truck). The new objective of the model is to maximise the probability of the correct samples coming from the corpus and minimise the corpus probability for the negative samples

https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5783648/

Wneg = {wk~Pw|k = 1, …, K} is the set of “negative words” that are sampled from a frequency distribution Pw over the word vocabulary W, where K is the number of randomly generated words for each context word. By maximizing (3), the dot product between frequently co-occurring words will become large and the dot product between rarely co-occurring words will become small.

https://rohanvarma.me/Word2Vec/

With NCE, word vectors are no longer learned by attempting to predict the context words from the target word. Instead we learn word vectors by learning how to distinguish true pairs of (target, context) words from corrupted (target, random word from vocabulary) pairs. The idea is that if a model can distinguish between actual pairs of target and context words from random noise, then good word vectors will be learned.

https://medium.com/explorations-in-language-and-learning/understanding-word-vectors-f5f9e9fdef98

For the word w, we are trying to predict the context word c. Since we are using softmax, this is essentially like a multi-class classification problem, where we are trying to classify the next word into one of N classes (where N is the number of words in the dictionary). Since N may be quite large, this is a very difficult problem.

What SGNS does is that it converts this multi-classification problem into binary classification. The new objective is to predict, for any given word-context pair (w,c), whether the pair is in the window or not. For this, we try to increase the probability of a “positive” pair (w,c), while at the same time reducing the probability of k randomly chosen “negative samples” (w,s) where s is a word not found in w’s context.

### Skip-Gram with Negative Sampling (SGNS)